In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the dataset for recommenders

In [2]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear',
             'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df['term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df['length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df['rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df['room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df['n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df['weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df['weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(interactions_df.head(15))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several versions of the prepare_users_df method and test which works best in your recommender.

In [3]:
interactions_df.head()

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True


In [4]:
interactions_df.groupby(['term']).count()['item_id']

term
WinterVacation    1452
Easter             168
OffSeason         6695
HighSeason        3223
LowSeason         2907
MayLongWeekend     337
NewYear            346
Christmas          140
Name: item_id, dtype: int64

In [5]:
interactions_df.groupby(['length_of_stay_bucket']).count()['item_id']

length_of_stay_bucket
[0-1]      2233
[2-3]      9063
[4-7]      3685
[8-inf]     287
Name: item_id, dtype: int64

In [6]:
interactions_df.groupby(['rate_plan']).count()['item_id']

rate_plan
Standard    7377
Nonref      7891
Name: item_id, dtype: int64

In [7]:
interactions_df.groupby(['n_people_bucket']).count()['item_id']

n_people_bucket
[1-1]      1045
[2-2]      5242
[3-4]      6717
[5-inf]    2264
Name: item_id, dtype: int64

In [8]:
interactions_df.groupby(['weekend_stay']).count()['item_id']

weekend_stay
True     10247
False     5021
Name: item_id, dtype: int64

In [9]:
hotel_df_pp = pd.read_csv('data/hotel_data/hotel_data_preprocessed.csv')
hotel_df_pp.head()

,Unnamed: 0,n_people,n_children_1,n_children_2,n_children_3,accommodation_price,meal_price,service_price,paid,n_rooms,...,email,phone,is_company,night_price,term,length_of_stay_bucket,room_segment,n_people_bucket,item,item_id
0,0,5,0,0,0,992.29,0.0,0.0,1.0,1,...,NaN,318faec979ecaf8adaee0c8e5d7531a67f309b7247d30b...,0,330.76,WinterVacation,[2-3],[260-360],[5-inf],WinterVacation [2-3] Standard [260-360] [5-inf...,0
1,1,4,0,0,0,693.40,0.0,0.0,693.4,1,...,f9c0564c66d6a830c4964a30ac261038dd7cf762b0641c...,cb550ba6d303bf230379073bcbdd55c37229eab3f173dc...,0,231.13,WinterVacation,[2-3],[160-260],[3-4],WinterVacation [2-3] Standard [160-260] [3-4] ...,1
2,2,2,0,0,0,366.80,0.0,0.0,1.0,1,...,f6a8c77530865b7e437eb746c3564c4cbdc522c10d35f6...,1c56315c10c9d8153ca7820648900befbd9109fb6cfb81...,0,183.40,WinterVacation,[2-3],[160-260],[2-2],WinterVacation [2-3] Standard [160-260] [2-2] ...,2
3,3,4,0,0,0,1064.60,0.0,0.0,1.0,1,...,6d08a7230580a09f1fde268bb7c1a5d74a55bdcc9183f8...,3aff5ce689580e51de899de8ec75e8a8eaa470e4e99df4...,0,152.09,WinterVacation,[4-7],[160-260],[3-4],WinterVacation [4-7] Standard [160-260] [3-4] ...,3
4,4,2,0,0,0,713.00,0.0,0.0,1.0,1,...,0d6aafda88cc3d5844da8c60ca9d1b6682f1ce1a4dfe12...,ea16c664798581a9d93a3128d772b8b89e05743edbbfae...,0,142.60,WinterVacation,[4-7],[0-160],[2-2],WinterVacation [4-7] Standard [0-160] [2-2] True,4


In [10]:
hotel_df_pp.groupby('user_id').mean()[['length_of_stay', 'n_people', 'night_price']]

,length_of_stay,n_people,night_price
user_id,,,
1,3.454545,2.454545,169.290455
2,3.000000,4.000000,231.130000
3,2.000000,2.000000,183.400000
4,7.000000,4.000000,152.090000
5,5.000000,2.000000,142.600000
...,...,...,...
14498,2.000000,1.000000,194.700000
14499,8.000000,2.000000,123.500000
14500,3.000000,4.000000,246.800000


In [11]:
def prepare_users_df(interactions_df):
    interactions = interactions_df.copy()

    room_segment_mapper = {
        '[0-160]': 80,
        '[160-260]': 220,
        '[260-360]': 320,
        '[360-500]': 430,
        '[500-900]': 700
    }
    length_of_stay_bucket_mapper = {
        '[0-1]': 0.5,
        '[2-3]': 2.5,
        '[4-7]': 6,
        '[8-inf]': 11
    }
    n_people_bucket_mapper = {
        '[1-1]': 1,
        '[2-2]': 2,
        '[3-4]': 3.5,
        '[5-inf]': 8
    }

    interactions['room_segment'] = interactions['room_segment'].replace(room_segment_mapper)
    interactions['length_of_stay_bucket'] = interactions['length_of_stay_bucket'].replace(length_of_stay_bucket_mapper)
    interactions['n_people_bucket'] = interactions['n_people_bucket'].replace(n_people_bucket_mapper)

    users = interactions.loc[:, ['user_id', 'room_segment', 'length_of_stay_bucket', 'n_people_bucket']].groupby(
        'user_id').mean()
    users = users.fillna(0).astype(int).reset_index()

    return users, ['user_id', 'room_segment', 'length_of_stay_bucket', 'n_people_bucket']


users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))

['user_id', 'room_segment', 'length_of_stay_bucket', 'n_people_bucket']


,user_id,room_segment,length_of_stay_bucket,n_people_bucket
0,1,233,3,2
40,50,263,2,4
84,96,222,2,3
102,115,270,4,4
371,706,225,3,3
1383,1736,226,2,1
7301,7779,220,6,4


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into one-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features, for instance based on numerical min and max values in buckets used as features.

In [12]:
interactions_df.head()

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True


In [13]:
def prepare_items_df(interactions_df):
    interactions = interactions_df.copy()
    interactions = interactions.drop_duplicates(subset='item_id')
    one_hot_encoded = pd.get_dummies(data=interactions,
                                     columns=['term', 'room_segment', 'rate_plan', 'length_of_stay_bucket',
                                              'n_people_bucket', 'weekend_stay'], prefix='item_feature')
    items_df = one_hot_encoded.drop(['user_id'], axis=1, errors='ignore')
    item_features = list(items_df.columns.values[1:])
    return items_df, item_features


items_df, item_features = prepare_items_df(interactions_df)

print(item_features)

display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))

['item_feature_WinterVacation', 'item_feature_Easter', 'item_feature_OffSeason', 'item_feature_HighSeason', 'item_feature_LowSeason', 'item_feature_MayLongWeekend', 'item_feature_NewYear', 'item_feature_Christmas', 'item_feature_[0-160]', 'item_feature_[160-260]', 'item_feature_[260-360]', 'item_feature_[360-500]', 'item_feature_[500-900]', 'item_feature_Standard', 'item_feature_Nonref', 'item_feature_[0-1]', 'item_feature_[2-3]', 'item_feature_[4-7]', 'item_feature_[8-inf]', 'item_feature_[1-1]', 'item_feature_[2-2]', 'item_feature_[3-4]', 'item_feature_[5-inf]', 'item_feature_True', 'item_feature_False']


,item_id,item_feature_WinterVacation,item_feature_Easter,item_feature_OffSeason,item_feature_HighSeason,item_feature_LowSeason,item_feature_MayLongWeekend,item_feature_NewYear,item_feature_Christmas,item_feature_[0-160],...,item_feature_[0-1],item_feature_[2-3],item_feature_[4-7],item_feature_[8-inf],item_feature_[1-1],item_feature_[2-2],item_feature_[3-4],item_feature_[5-inf],item_feature_True,item_feature_False
0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
1,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,2,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
3,3,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
4,4,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
5,5,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
6,6,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object as self.users_df for later use in the recommend method. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real ("positive") interactions and equal to 0 for those newly added "negative" interactions. Generate several negative interactions per every positive interaction (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, LightGBM, Neural Networks or any model of your choice.

In [14]:
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso
from sklearn.svm import SVR
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos

        self.recommender_df = pd.DataFrame(
            columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None

        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)

    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.

        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """

        interactions_df = interactions_df.copy()

        # Prepare users_df and items_df

        users_df, user_features = prepare_users_df(interactions_df)

        self.users_df = users_df
        self.user_features = user_features

        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]

        # Generate negative interactions

        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]

        interactions_df.loc[:, 'interacted'] = 1

        ########################
        # Write your code here #
        # Generate tuples (user_id, item_id, 0) for pairs (user_id, item_id) which do not
        # appear in the interactions_df and add those tuples to the list negative_interactions.
        # Generate self.n_neg_per_pos * len(interactions_df) negative interactions
        # (self.n_neg_per_pos per one positive).
        # Make sure the code is efficient and runs fast, otherwise you will not be able to properly tune your model.
        negative_interactions = []

        pos_tuples = interactions_df.loc[:, ['user_id', 'item_id']]

        pos_tuples = set(pos_tuples.itertuples(index=False, name=None))
        pos_tuples = {key: 1 for key in pos_tuples}

        user_ids, item_ids = interactions_df.user_id.unique(), interactions_df.item_id.unique()

        negative_interactions_count = self.n_neg_per_pos * len(interactions_df)
        while len(negative_interactions) < negative_interactions_count:
            random_user_id = np.random.choice(user_ids)
            random_item_id = np.random.choice(item_ids)

            if (random_user_id, random_item_id) not in pos_tuples:
                negative_interactions.append((random_user_id, random_item_id, 0))

        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])

        # Get the input data for the model

        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])

        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values

        self.model.fit(x, y)

    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns
        top n_recommendations for each user.

        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations
            for each user.
        :rtype: pd.DataFrame
        """

        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]

        ########################
        # Write your code here #
        # Prepare users_df and items_df
        # For users_df you need to merge user features from self.users_df to users_df
        # (the users for which you generate recommendations).
        # Note that for users who were not in the hotel before (which is true for most users)
        # there will be no features in self.users_df. For such users you can initialize their features
        # with all zeros (for instance with fillna(0)), but you can also try to use average feature
        # values from self.users_df (this way you would trear a new user as an average user).
        # For items you have to apply the prepare_items_df method to items_df.

        users_df = pd.merge(users_df, self.users_df, on=['user_id'], how='left').fillna(0)
        items_df, item_features = prepare_items_df(items_df)

        for ix, user in users_df.iterrows():

            ########################
            # Write your code here #
            # Create a Carthesian product of users from users_df and items from items_df
            carthesian_product = pd.merge(items_df, user.to_frame().transpose(), how='cross')

            ########################
            # Write your code here #
            # Use self.model.predict method to calculate scores for all records in the just created DataFrame
            # of users and items
            scores = self.model.predict(carthesian_product[user_features + item_features].values)

            ########################
            # Write your code here #
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.

            chosen_ids = (-scores).argsort()[:n_recommendations]

            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[item_id]
                    }
                )

            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df


class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()


class RidgeRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Ridge regression recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = Ridge(**model_params)


class LassoRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Lasso regression recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = Lasso(**model_params)


class ElasticNetRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Ridge regression recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = ElasticNet(**model_params)


class BernoulliNaiveBayesCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Bernoulli Naive Bayes recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = BernoulliNB()


class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)


class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)


class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)

# Quick test of the recommender

In [18]:
cb = ContentBasedUserItemRecommender()
cb.fit(interactions_df, users_df, items_df)

In [16]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

# Static parameters

In [38]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

# Tuning method

In [39]:
from hyperopt import hp, fmin, tpe, Trials
import traceback


def tune_recommender(recommender_class, interactions_df, items_df,
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest,
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1

    if not succeded:
        return None

    # Validate

    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results,
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5',
                                    'NDCG@10'])

    display(results)

    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [186]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 30, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 10/10 [14:16<00:00, 85.62s/trial, best loss: -0.2293694456199746]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.243381,0.049219,0.094502,0.113016,0.134965


Best parameters:
{'n_neg_per_pos': 1.0}


In [187]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 15, 1),
    'alpha': hp.quniform('alpha', 0.1, 100, 1),
}

best_param_set = tune_recommender(RidgeRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 10/10 [14:58<00:00, 89.88s/trial, best loss: -0.22683030046550995]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RidgeRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.23184,0.049219,0.094502,0.113016,0.131421


Best parameters:
{'alpha': 33.0, 'n_neg_per_pos': 7.0}


In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  1%|          | 1/100 [1:16:30<126:14:54, 4590.85s/trial, best loss: -0.2331781633516716]

In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [ ]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

## Amazon

In [49]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results,
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(amazon_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.044128,0.118805,0.161236,0.222335,0.044128,0.086755,0.104479,0.12421


## Linear Regression

In [188]:
cb_user_item_recommender = LinearRegressionCBUIRecommender(n_neg_per_pos=1)
linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results,
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.244399,0.049219,0.094502,0.113016,0.135231


## Ridge Regression

In [189]:
cb_user_item_recommender = RidgeRegressionCBUIRecommender(n_neg_per_pos=7,
                                                          alpha=33.0)  # Initialize your recommender here with the best params from tuning
# Give the name of your recommender in the line below
ridge_cbui_tts_results = [['RidgeRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

ridge_cbui_tts_results = pd.DataFrame(
    ridge_cbui_tts_results,
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(ridge_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RidgeRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.23184,0.049219,0.094502,0.113016,0.131421


## Lasso Regression

In [64]:
cb_user_item_recommender = LassoRegressionCBUIRecommender(
    alpha=0.1)  # Initialize your recommender here with the best params from tuning
# Give the name of your recommender in the line below
lasso_cbui_tts_results = [['LassoRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

lasso_cbui_tts_results = pd.DataFrame(
    lasso_cbui_tts_results,
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(lasso_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LassoRegressionCBUIRecommender,0.003394,0.004073,0.004752,0.005092,0.003394,0.003778,0.004056,0.004154


## SVR

In [18]:
cb_user_item_recommender = SVRCBUIRecommender()  # Initialize your recommender here with the best params from tuning
# Give the name of your recommender in the line below
svr_cbui_tts_results = [['SVRCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

svr_cbui_tts_results = pd.DataFrame(
    svr_cbui_tts_results,
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(svr_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,SVRCBUIRecommender,0.049219,0.133401,0.179905,0.242702,0.049219,0.097266,0.116104,0.136182


## Bernoulli Naive Bayes

In [56]:
cb_user_item_recommender = BernoulliNaiveBayesCBUIRecommender()  # Initialize your recommender here with the best params from tuning
# Give the name of your recommender in the line below
bernoulli_nb_cbui_tts_results = [['BernoulliNaiveBayesCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

bernoulli_nb_cbui_tts_results = pd.DataFrame(
    bernoulli_nb_cbui_tts_results,
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(bernoulli_nb_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,BernoulliNaiveBayesCBUIRecommender,0.001358,0.009504,0.016972,0.033605,0.001358,0.005876,0.008973,0.014341


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,BernoulliNaiveBayesCBUIRecommender,0.000679,0.007128,0.01799,0.033265,0.000679,0.004392,0.008922,0.013708


## Random Forest

In [57]:
cb_user_item_recommender = RandomForestCBUIRecommender()
# Give the name of your recommender in the line below
random_forest_cbui_tts_results = [['RandomForestCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

random_forest_cbui_tts_results = pd.DataFrame(
    random_forest_cbui_tts_results,
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(random_forest_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.011881,0.01799,0.022743,0.034284,0.011881,0.01538,0.017367,0.021135


## XGBoost

In [26]:
cb_user_item_recommender = XGBoostCBUIRecommender()
# Give the name of your recommender in the line below
xgboost_cbui_tts_results = [['XGBoostCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

xgboost_cbui_tts_results = pd.DataFrame(
    xgboost_cbui_tts_results,
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])
display(xgboost_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.000339,0.001018,0.001018,0.001697,0.000339,0.000723,0.000723,0.000951
